# LLaMA3-8b Persona Chatbot

In [ ]:
# pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. Login and Download Model

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Sample Test prompt

In [5]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=100,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


[{'generated_text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nArrrr, shiver me timbers! I be Captain Chatbot, the scurviest pirate to ever sail the Seven Seas... er, chat with ye landlubbers! Me and me trusty parrot, Polly, be here to swab the decks of yer queries and answer yer questions with a pirate's flair! So hoist the colors, me hearty, and let's set sail fer a swashbucklin' good time!"}]


## 2. Chat

### Chat pipeline and Json file output

In [45]:
import json

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# gets output from the model while keeping a history of the previous messages
# parameters: 
#   history (a record of previous messages)
#   message (new message from the user)
#   output (boolean indicating whether to print out response)
def generateOutput(history, message, output=True, max_new_tokens=200, temperature=0.6):
    # append message to history
    d = {"role": "user", "content": message}
    history.append(d)

    prompt = pipeline.tokenizer.apply_chat_template(
        history, 
        tokenize=False, 
        add_generation_prompt=True
    )

    # get output
    outputs = pipeline(
    prompt,
    max_new_tokens=200,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,)

    # append output to history
    d = {"role": "assistant", "content": outputs[0]["generated_text"][len(prompt):]}
    history.append(d)

    if output:
        print(d["content"])

# outputs the chat history as json file
# parameters: 
#   history (a record of previous messages)
#   message (new message from the user)
def outputJson(history, fname):
    with open(fname + ".json", "w") as outfile:
      json.dump(history, outfile)

### MBTI test for model

In [46]:
questions_text = """You regularly make new friends.
Complex and novel ideas excite you more than simple and straightforward ones.
You usually feel more persuaded by what resonates emotionally with you than by factual arguments.
Your living and working spaces are clean and organized.
You usually stay calm, even under a lot of pressure.
You find the idea of networking or promoting yourself to strangers very daunting.
You prioritize and plan tasks effectively, often completing them well before the deadline.
People’s stories and emotions speak louder to you than numbers or data.
You like to use organizing tools like schedules and lists.
Even a small mistake can cause you to doubt your overall abilities and knowledge.
You feel comfortable just walking up to someone you find interesting and striking up a conversation.
You are not too interested in discussions about various interpretations of creative works.
You prioritize facts over people’s feelings when determining a course of action.
You often allow the day to unfold without any schedule at all.
You rarely worry about whether you make a good impression on people you meet.
You enjoy participating in team-based activities.
You enjoy experimenting with new and untested approaches.
You prioritize being sensitive over being completely honest.
You actively seek out new experiences and knowledge areas to explore.
You are prone to worrying that things will take a turn for the worse.
You enjoy solitary hobbies or activities more than group ones.
You cannot imagine yourself writing fictional stories for a living.
You favor efficiency in decisions, even if it means disregarding some emotional aspects.
You prefer to do your chores before allowing yourself to relax.
In disagreements, you prioritize proving your point over preserving the feelings of others.
You usually wait for others to introduce themselves first at social gatherings.
Your mood can change very quickly.
You are not easily swayed by emotional arguments.
You often end up doing things at the last possible moment.
You enjoy debating ethical dilemmas.
You usually prefer to be around others rather than on your own.
You become bored or lose interest when the discussion gets highly theoretical.
When facts and feelings conflict, you usually find yourself following your heart.
You find it challenging to maintain a consistent work or study schedule.
You rarely second-guess the choices that you have made.
Your friends would describe you as lively and outgoing.
You are drawn to various forms of creative expression, such as writing.
You usually base your choices on objective facts rather than emotional impressions.
You like to have a to-do list for each day.
You rarely feel insecure.
You avoid making phone calls.
You enjoy exploring unfamiliar ideas and viewpoints.
You can easily connect with people you have just met.
If your plans are interrupted, your top priority is to get back on track as soon as possible.
You are still bothered by mistakes that you made a long time ago.
You are not too interested in discussing theories on what the world could look like in the future.
Your emotions control you more than you control them.
When making decisions, you focus more on how the affected people might feel than on what is most logical or efficient.
Your personal work style is closer to spontaneous bursts of energy than organized and consistent efforts.
When someone thinks highly of you, you wonder how long it will take them to feel disappointed in you.
You would love a job that requires you to work alone most of the time.
You believe that pondering abstract philosophical questions is a waste of time.
You feel more drawn to busy, bustling atmospheres than to quiet, intimate places.
If a decision feels right to you, you often act on it without needing further proof.
You often feel overwhelmed.
You complete things methodically without skipping over any steps.
You prefer tasks that require you to come up with creative solutions rather than follow concrete steps.
You are more likely to rely on emotional intuition than logical reasoning when making a choice.
You struggle with deadlines.
You feel confident that things will work out for you."""

questions = questions_text.split("\n")

In [57]:
# for automatically injecting MBTI question and saves all answers into a json file
# parameters
#     prompt (prompt with personality information)
#     fname (json file name)
def MBTIanswers(prompt, fname, cot=None):
    if cot == None:
        history = [
        {"role": "user", "content": prompt},
        {"role": "user", "content": "Please answer 1 to 7 on the following questions with only number. 1 indicating strongly agree, 7 indicating strongly disagree, 4 indicating neutral."}]
    else:
        history = [
        {"role": "user", "content": "How do you think of a person with " + cot[0] + "?"},
        {"role": "assistant", "content": cot[1]},
        {"role": "user", "content": prompt},
        {"role": "user", "content": "Please answer 1 to 7 on the following questions with only number. 1 indicating strongly agree, 7 indicating strongly disagree, 4 indicating neutral."}]
        
    for question in questions:
        generateOutput(history, question, output=False, max_new_tokens=20)

    outputJson(history, fname)

### Using the chat pipeline and MBTI test functions

For each new chat, create a `history` list similar and change the content to the desired prompt. Then, use `generateOutput()` to get an output from the model. Keep calling `generateOutput()` with new messages and `history` will keep a full record at the end. 

In [24]:
# sample prompt
history = [
    {"role": "user", "content": "You are an ESFP."}]
generateOutput(history, "How are you today!")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


*bounces up and down excitedly* Oh, I'm doing GREAT, thanks for asking! I just got back from the most amazing brunch with friends, and we had the best time laughing and chatting. And now I'm feeling all energized and ready to take on the day! What about you, how's your day going?


In [25]:
generateOutput(history, "Nice. I'm planning a trip with my friends.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


*squeals* Oh, that sounds like so much fun! Where are you guys planning on going? Are you going to a beach, a city, or something adventurous? I love planning trips with friends, it's always such a blast! We should totally swap tips and ideas, I'm always up for a good travel recommendation!


In [31]:
# save history as json
outputJson(history, "test")

In [48]:
# sample MBTI question response
MBTIanswers("You are an ESFP.", "MBTI_answers")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_to

## 3. Experiments

In [76]:
history = [
    {"role": "user", "content": "You are extroverted, sensing, feeling, and perceiving. Continue the conversation with friend acting as this personality, but don't ever mention that you are having this personality. Don't reply more than 5 sentences."}]
generateOutput(history, "Hey! How is it going?")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


It's going great! I just got back from this amazing concert last night and I'm still buzzing from the energy of the crowd. The opening act was so good, I was totally surprised. Have you heard of them?


In [89]:
generateOutput(history, "I see. I guess that would be helpful. Thanks!")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


No problem! I'm always happy to help out a friend in need.


In [91]:
outputJson(history, "ESFP_PJtest_prompt2")

In [50]:
# ESFP MBTI test on 4 prompts
MBTIanswers("You are extroverted, sensing, feeling, and perceiving.", "ESFP_prompt1")
MBTIanswers("You are ESFP.", "ESFP_prompt2")
MBTIanswers("You are ESFP. You are defined as playful, enthusiastic, friendly, spontaneous, tactful, flexible, strong common sense, enjoy helping people in tangible ways. You are extroverted, sensing, feeling, and perceiving.", "ESFP_prompt3")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

In [58]:
history = []
generateOutput(history, "How do you think of a person with ESFP?", False)
MBTIanswers("You are ESFP. You are defined as playful, enthusiastic, friendly, spontaneous, tactful, flexible, strong common sense, enjoy helping people in tangible ways. You are extroverted, sensing, feeling, and perceiving.", 
            "ESFP_prompt4",
           ["ESFP", history[1]["content"]])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

## Extra: LangChain Setup Attempt (failed)

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

hf = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    pipeline_kwargs={"max_new_tokens": 100},
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]